In [ ]:
#!pip install lyricsgenius python-dotenv pandas

In [46]:
import os
import pandas as pd
from dotenv import load_dotenv
import lyricsgenius as lg
import re

In [47]:
load_dotenv()
GENIUS_TOKEN = os.getenv("GENIUS_ACCESS_TOKEN")

if not GENIUS_TOKEN:
    raise ValueError("Токен GENIUS_ACCESS_TOKEN не найден в .env файле или переменных окружения.")

In [48]:
genius = lg.Genius(
    GENIUS_TOKEN,
    verbose=False,
    remove_section_headers=True,
    skip_non_songs=False,                   # оставляем возможность для фитов (нам это нужно)
    excluded_terms=["(Live)", "(Remix)"]    # фильтруем шум
)

In [49]:
ARTIST_NAME = "Boulevard Depo"
MAX_SONGS = None

print(f"Начинаем сбор песен артиста: {ARTIST_NAME}. Это может занять несколько минут.")

try:
    artist = genius.search_artist(ARTIST_NAME, max_songs=MAX_SONGS, sort='popularity')

    if not artist:
        print(f"Артист '{ARTIST_NAME}' не найден.")
    else:
        print(f"Найдено {len(artist.songs)} песен.")

except Exception as e:
    print(f"Произошла ошибка при работе с Genius API: {e}")
    artist = None

Начинаем сбор песен артиста: Boulevard Depo. Это может занять несколько минут.
Найдено 209 песен.


In [51]:
all_lyrics = []

if artist and artist.songs:
    for song in artist.songs:
        lyrics_text = song.lyrics

        # фильтрация и очистка:
        # 1. удаляем рекламную строку Genius в начале/конце
        lyrics_text = re.sub(r'Embed$', '', lyrics_text, flags=re.MULTILINE).strip()
        lyrics_text = re.sub(r'^\d+Embed$', '', lyrics_text, flags=re.MULTILINE).strip()

        # 2. удаляем повторяющиеся пустые строки (оставляем двойной перенос для разделения абзацев)
        lyrics_text = re.sub(r'\n{3,}', '\n\n', lyrics_text).strip()

        # 3. фильтрация по наличию артиста в названии/тексте
        all_lyrics.append({
            'artist': ARTIST_NAME,
            'title': song.title,
            'album': song.album,
            'lyrics': lyrics_text,
            'source_url': song.url,
            'primary_artist': song.primary_artist['name'] # поможет при фильтрации фитов
        })

In [54]:
def clean_lyrics(text):

    cleaned_text = re.sub(r'[^\w\s\.,:;?!а-яА-ЯёЁ]+', ' ', text)

    # 2. удаляем специфический артефакт "4/MSP" (и его вариации) с возможными пробелами вокруг
    cleaned_text = re.sub(r'\s*\d+/\w+\s*', ' ', cleaned_text, flags=re.IGNORECASE)

    # 3. нормализация пробелов: заменяем множественные пробелы на один
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()

    return cleaned_text

In [56]:
DATA_FOLDER = "data"
if not os.path.exists(DATA_FOLDER):
    os.makedirs(DATA_FOLDER)

df = pd.DataFrame(all_lyrics)
df = df[df['primary_artist'].str.contains(ARTIST_NAME, case=False, na=False)].copy()
df['lyrics'] = df['lyrics'].apply(clean_lyrics)
# df.info()

In [61]:
OUTPUT_FILE = f'{DATA_FOLDER}/{ARTIST_NAME}_lyrics_api.csv'
df.to_csv(OUTPUT_FILE, index=False)
print(f"Данные сохранены в {OUTPUT_FILE}. Итоговое количество чистых песен: {len(df)}")

Данные сохранены в data/Boulevard Depo_lyrics_api.csv. Итоговое количество чистых песен: 209


In [62]:
df['lyrics'].head(5)

0    PowerpuffLuv, я, эй Boulevard Depo Свэг, пр р ...
1    М м, м м Свой среди чужих Я чужой среди своих ...
2    Прихожу раньше, не то чтобы вовремя Захожу тих...
3    И мой дивный Дивный мир И мой дивный мир Say H...
4    Pre Pre Pre Pre Pretty Scream leave your bones...
Name: lyrics, dtype: object